# First Pass Naive Bayes Model on Text Data

In [2]:
from pymongo import MongoClient
import pymongo
import os
import pandas as pd
import pickle
import numpy as np
import re
from urllib.parse import urlparse

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import naive_bayes
from sklearn.pipeline import Pipeline

In [4]:
# password = os.environ['password']
# libera_url = "54.89.159.106"

# client = pymongo.MongoClient("mongodb://paul:" + password + "@" + libera_url + "/libera_db")

# db = client.libera_db

# col = db.scraped_blogs

# cursor = col.find({},{'_id':1,'text':1,'url':1,'quality':1})

# df = pd.DataFrame(list(cursor))

# pickle.dump( df, open( "mongodb_data.p", "wb" ) )

In [135]:
x = df.query('url == "https://chennaifocus.in/"')

In [136]:
x

,_id,quality,text,url,predicted
1364,593f4834f7e49c1f7a2904c1,False,"b'b""Skip to navigation Skip to main content Sk...",https://chennaifocus.in/,True


In [143]:
df['text'][1364]

'b\'b"Skip to navigation Skip to main content Skip to primary sidebar Skip to secondary sidebar Skip to footer Chennai Focus     Information About Chennai Zoom in on Chennai Home ChennaiFocus     We Are Because You Are    Blog Disclosure Statement Contact Us Love Chennai Focus Pitch Video YouTube Twitter Facebook RSS Feed Welcome to Era of Easy Communication and Misleading  Information Jun  Posted by Katie Nathan The title says it all, we are living in an exalted era of communication and being fed with information sometimes authentic but many unverified and unproven. To add to this, we have the power to pass it on and it goes undeterred and unquestioned to gullible people who never question the authenticity of the information given to them. The reach is even more when it touches the life of a common man. One such case doing the rounds is    cPlastic Rice   d. We also got a video in WhatsApp about Chinese plastic rice entering the market and how it is manufactured. Smartly, there was no

In [139]:
df.iloc[1364,:]['text']

'b\'b"Skip to navigation Skip to main content Skip to primary sidebar Skip to secondary sidebar Skip to footer Chennai Focus     Information About Chennai Zoom in on Chennai Home ChennaiFocus     We Are Because You Are    Blog Disclosure Statement Contact Us Love Chennai Focus Pitch Video YouTube Twitter Facebook RSS Feed Welcome to Era of Easy Communication and Misleading  Information Jun  Posted by Katie Nathan The title says it all, we are living in an exalted era of communication and being fed with information sometimes authentic but many unverified and unproven. To add to this, we have the power to pass it on and it goes undeterred and unquestioned to gullible people who never question the authenticity of the information given to them. The reach is even more when it touches the life of a common man. One such case doing the rounds is    cPlastic Rice   d. We also got a video in WhatsApp about Chinese plastic rice entering the market and how it is manufactured. Smartly, there was no

In [133]:
x['text']

1398    b'b\'The SqlGrrrl Isabelle Van Campenhoudt >>>...
Name: text, dtype: object

In [3]:
df = pickle.load(open("mongodb_data.p",'rb'))

In [3]:
reference = pickle.load(open("mongodb_data.p",'rb'))

In [4]:
reference.shape

(3813, 4)

#### Preliminary Data Cleaning

In [302]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3813 entries, 0 to 3812
Data columns (total 4 columns):
_id        3813 non-null object
quality    3813 non-null bool
text       3813 non-null object
url        3813 non-null object
dtypes: bool(1), object(3)
memory usage: 93.2+ KB


In [4]:
df['quality'].value_counts()

False    2836
True      977
Name: quality, dtype: int64

In [160]:
def clear_empty_text(text):
    if len(text) < 1:
        text = np.NaN
    return text

In [161]:
def get_netloc(url):
    netloc = urlparse(url).netloc
    return netloc

In [162]:
comment_pattern = re.compile(r'#comment',re.I)
tag_category_pattern = re.compile(r'/tag/|/category/', re.I)

In [163]:
fake_url = 'http://www.test.com/category/list'
edge_case = 'http://www.test.com/tag/list'

In [164]:
def remove_comment_pages(url):
    if url == np.NaN or comment_pattern.search(url):
        url = np.NaN
    return url

In [165]:
def remove_tag_category_pages(url):
    if url == np.NaN or tag_category_pattern.search(url):
        url = np.NaN
    return url

In [166]:
def remove_java(text):
    java_pattern = re.compile(r'$.+|.+\..+|.*\{.*|.*\}.*|.*\);', re.DOTALL)
    new_text = ''
    for word in text.split(' '):
        if java_pattern.search(word):
            word = ''
        else:
            word = word
        new_text += word + ' '
    return new_text

In [167]:
def remove_numbers(text):
    number_pattern = re.compile(r'[0-9_]+')
    new_text = ''
    for word in text.split(' '):
        if number_pattern.search(word):
            word = ''
        else:
            word = word
        new_text += word + ' '
    return new_text

In [168]:
# def remove_leftover_javascript(text):
# #     text = text.encode(encoding='ASCII',errors='replace')
#     java_pattern = re.compile(r'\{.*\}',re.DOTALL)
#     re.sub(java_pattern, ' ', text)
#     return text

In [169]:
def remove_encoded(text):
    encoding_patterns = re.compile(r'\\xc[0-9]+|\\xe[0-9]+|\\x[0-9]+|\\xa[0-9]+|\\xb[0-9]+')
    cleaned_text = re.sub(encoding_patterns, ' ', str(text.encode()))
    return cleaned_text

In [170]:
df['text'] = df['text'].apply(remove_numbers)

df['text'] = df['text'].apply(remove_java)

df['text'] = df['text'].apply(remove_encoded)

df['text'] = df['text'].apply(clear_empty_text);

df['url'] = df['url'].apply(remove_comment_pages)

df.dropna(inplace=True)
df.reset_index(inplace=True)

df.drop('index',axis=1,inplace=True)

# df['url'] = df['url'].apply(remove_tag_category_pages)
# df.dropna(inplace=True)

# df.reset_index(inplace=True)

# df.drop('index',axis=1,inplace=True)

In [178]:
# df['url'] = df['url'].apply(remove_tag_category_pages)
# df.dropna(inplace=True)

In [75]:
# df['netloc'] = df['url'].apply(get_netloc)

In [76]:
df['quality'].value_counts()

False    2836
True      977
Name: quality, dtype: int64

In [255]:
# withheld = df.query('netloc == "www.kaushik.net"')

# df = df.query('netloc != "www.kaushik.net"')

# df.reset_index(inplace=True)
# df.drop('index',axis=1,inplace=True)
# withheld.reset_index(inplace=True)
# withheld.drop('index',axis=1,inplace=True)

In [256]:
reference['quality'].value_counts()

False    2836
True      977
Name: quality, dtype: int64

#### Tokenization and Vectorization

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

import string

In [118]:
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

stopword = stopwords.words('english')
stopword += list(string.punctuation)
stopword += ['``', '\'\'', '–','.','=',')','(',';','``']
stopword += ['january','february','march','april','may','june','july','august','september','october','november',
            'december','jan','feb','mar','apr','jul','aug','sep','oct','nov','dec', 'fadeoutdown',
             'fadeinup','getelementsbyclassname','serif','sans','answer','mon','tue','wed','thu','fri','sat','sun',
            'br','content','cdata','neue', 'helvetica','faq','feedplaceholdergroup','var','getelementsbyclassname',
             'js','parseparamsfromurl','function','follow','like','mouseout','need','share','tweet','email',
             'address', 'name', 'website','keyvaluepair','nvar','onloadfunctionsobj', 'createscripttagfunc',
            'subscribe','rss','queryparamname', 'urlparams','cbsigptdivids','np', 'xef', 'xbf', 'ntext',
             'numsources', 'encryptedtoken','usessl', 'https', 'promotion', 'gs', 'visibleurl', 'qe', 'xef', 'xbf',
            'searchclick', 'customprovider','customprovider','cx','ie','dtick','whitelabelsharing','setinterval',
             'tjquery','fileformattype','hover','buttonseffect','imageresult','span','customsearchoptions', 'th',
             'customsearchcontrol','clearinterval','queryparamname','want','webresult','isstorifyurl','arial',
             'border', 'color','background','blog', 'receive','notif','browser', 'facebook','check','font','display',
            'comment','commentform','contact','twitter','family','fieldset','googletag','permalink']

def tokenize(text_column, tokenizer=TreebankWordTokenizer(), stemmer=PorterStemmer()):
    """Tokenizes and stems words in a text doc
    Uses TreebankTokenizer and Porter Stemmer by default
    
    This function also removes words with numbers and underscores,
    eliminates punctuation, and returns all the words as lowercase
    """
    token_list = []
    for row in text_column:
        row = row.lower()
        pattern = re.compile('[^a-zA-Z-\' ]+|x\w{2,6}')
        tokens = tokenizer.tokenize(row)
        tokens_no_stopwords = [w for w in tokens if w not in stopword]
        tokens_dropped_patterns = [w for w in tokens_no_stopwords if not pattern.search(w)]
        stems  = stem_tokens(tokens_dropped_patterns, stemmer)
        blogs= ' '.join(stems)
        token_list.append(blogs)
    return token_list

def stem_tokens(tokens, stemmer):
    """Stems tokenized words, this function is called by tokenize which uses Porter Stemmer by default"""
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [125]:
tokenize('hello my name is stupid')

'hello stupid'

In [262]:
tokens = tokenize(df['text'])

In [263]:
pickle.dump( tokens, open( "tokenized_text.p", "wb" ) )

START HERE!  
Retry different ngram ranges! Max Features Settings! Whatever it takes, then when you're set with your final model retry witholding that blog and wham bam zam live scraping!


In [29]:
tokens = pickle.load(open("tokenized_text.p", "rb"))

In [51]:
vect = CountVectorizer(stop_words=stopword, ngram_range=(2,3), max_features=5000)

In [52]:
tfidf_trans = TfidfTransformer()

In [53]:
vectorized_tokens = tfidf_trans.fit_transform(vect.fit_transform(tokens))

In [16]:
# pickle.dump(tfidf_trans, open("tfidf_trans.p","wb"))

In [148]:
vect = pickle.load(open("count_vect.p","rb"))

In [146]:
naivebayes = pickle.load(open("fit_naive_bayes.p","rb"))

In [147]:
tfidf_trans = pickle.load(open("tfidf_trans.p","rb"))

In [150]:
example_blog_post = 'b\'b"Skip to navigation Skip to main content Skip to primary sidebar Skip to secondary sidebar Skip to footer Chennai Focus     Information About Chennai Zoom in on Chennai Home ChennaiFocus     We Are Because You Are    Blog Disclosure Statement Contact Us Love Chennai Focus Pitch Video YouTube Twitter Facebook RSS Feed Welcome to Era of Easy Communication and Misleading  Information Jun  Posted by Katie Nathan The title says it all, we are living in an exalted era of communication and being fed with information sometimes authentic but many unverified and unproven. To add to this, we have the power to pass it on and it goes undeterred and unquestioned to gullible people who never question the authenticity of the information given to them. The reach is even more when it touches the life of a common man. One such case doing the rounds is    cPlastic Rice   d. We also got a video in WhatsApp about Chinese plastic rice entering the market and how it is manufactured. Smartly, there was no audio to it. We could see raw material (glue) kind made into plastic sheets and then being shaped like rice and collected in bags. My first thought process was shock but then a bit prudence set in as I remembered the process of making thermocol where sago like bubbles are manufactured and made into blocks. It may be the    cplastic rice   d is a raw material for some industrial product. Where is the proof that it is being sold as rice? Sadly, Basmati rice, nature   s gift to India with its aroma and flavor has come under question putting so many farmer lives and produce under threat. Being an ardent Basmati rice fan, I set about checking various trustworthy news about it. Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Editors Corner , Food Leave a comment Tags: awareness , food , India , plastic rice , Rice , viral videos Review of House of Cards Season   Jun  Posted by Katie Nathan House of Cards show focuses on the life of the Underwood, Frank and Claire as they foray into politics. From being a senator from North Carolina to getting promoted to Vice President and there to President. Frank, an unconventional man uses his unorthodox ways to get the better of them and steps his way up and kills any man or woman who gets in the way of his uprising. This show elucidates and gives the viewers an insight to American Politics from the perspective of a very strong man, Frank. This show does focus on other people such as Claire, Frank   s wife on how she helps her husband rise to power.    cThe power couple   d is what they are known to be as  they use politics and business hand in hand to rise in power, to prove power is worthier than money in this world. Frank Underwood is coming to India with the television premiere of House of Cards Season  on Saturday,  June,  PM onwards, only on Zee Caf   ! This season starts on a strong note with Frank becoming President but we should wait and see if the power in the show is retained, specifically the anti-climax plot that creates the much needed excitement. This season will hopefully be better than last season as Season  became a bit dull after Frank becomes President. However, since this series starts off with Frank being his own self, it sends a strong note  and the fans have even more to rejoice that Frank Underwood continues to rise to penultimate power and authority and does what he does best even though he is the most powerful man in the free world. This first episode posts a satirical note on the Government and how it can get awry and complicated and if justice is done, there will be a positive note left by the people to the President and support is just a matter of belief. This episode gives a ravaging lift off to Season  as all fans know, Frank Underwood cannot be changed and everyone should know it (according to Frank). The first episode brings up a positive note to fans and gives a shining example of how Mr. Underwood retains his glamour in a more older style. If you could not care more for political drama in this world, this episode fits right in.           Spoilers Ahead           Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Entertainment Leave a comment Tags: #HOConZCafe , House of Cards , review , Season  , Television Serial , ZCafe , ZEE cafe Mango Madness at The Forum Vijaya  Mall May  Posted by Katie Nathan Summer time, the king of fruits makes its grand entry in many forms. Needless to say about the varieties of mangoes we are familiar with, dishes that celebrate mangoes find a place in the menu of every restaurant in the city during this  With the advent of the concept of food courts, people of varied taste find it easy to get together to taste the food of their choice under one shelter. Forum Vijaya Mall boasts of multi cuisine food joints in the food court and many of them have curated special items to bring the best of the seasonal  We were invited for a tasting session at the Forum Mall Food Court to taste the #MangoMadness currently on which extends till June  We had the pleasure of tasting vegetarian dishes and it was interesting to see the array of innovative dishes from the participating outlets such as Kailash Parbat, Amaravathi, Kobe sizzlers,   Dashing Dhaba, Quiznows    Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Food , Restaurants Leave a comment Tags: chennai , food , food court , foodie , forum mall , Mango , Mango based dishes , Summer special , vegetarian , vijaya forum mall Copper Chimney   s The Great Indian Kabab Festival     Dig into Authentic Traditional  Food Mar  Posted by Katie Nathan Authentic! Traditional! That  touch! Old street taste, inviting aroma     Few phrases we use to locate that specific food that will take us back to those golden days when each dish with that one common main ingredient tasted so different and unique. Those days, the chef signature was imbibed in everything and whatever we try at home, we will go back to him for that specific  These days, most of the times when we eat out, we end up feeling better making the dish at home and let me tell you, at times it is much better. Take for example, tomato soups at restaurants taste almost like Maggi soups we make at home. Where is that signature that makes us drool and walk in to the restaurant for that experience? We happily had one such experience with Copper Chimney at  The Great Indian Kabab Festival happening now  till April  th  Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Food , Restaurants Leave a comment Tags: chennai , copper chimney , food , food blogger , food festival , kabab , restaurant review , traditional food , vegetarian Travel Diary      Travel to Madhya Pradesh Jyotir Lingams      Ujjain Mar  Posted by Katie Nathan We had a long pending visit to Ujjain and Omkareshwar and since young when I studied history, Ujjain has always enamored  So finally, we decided to make it and our travel began with train tickets to Bhopal and travel in car from there on a round trip to Ujjain and Omkareshwar. One hard lesson we learnt is never trust cab services of Ola or Uber on a weekend. It took  minutes for us to get a vehicle to Madras Central and thankfully for clear roads and green signals, we would not have made it to our train on time. We have been hearing so much about better services in long distance trains and to utter disappointment, Tamilnadu Express food remained the same as we have seen few years back. The only solace was cleaner toilets. We reached Bhopal at  PM the next day and stayed overnight at our friend   s place who won us over with their hospitality.   Our friend suggested that we visit Kharjana Ganesh Temple in Indore on the way to Ujjain. The temple is maintained by volunteers and is a very famous temple in the area. Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Stay , Travel  Comments Tags: India , Jyotirlinga , Mahakaleshwar , Shaktipeet , Spiritual Journey , travel , Ujjain , Visit Ujjain Following Tea Trails in Chennai     Let us Eat and Drink  Tea Mar  Posted by Katie Nathan Chennai known more for its affinity to Coffee or kaapi is now getting its share of tea tasting experience with exclusive outlets for getting that Tea experience. Tea is no more limited to the hot beverage you get in roadside shops with a huge bag like filter and served in glasses. Tea Trails located in Ashok Nagar near the police station serves  different types of tea categorized as black, white, green, Oolong and herbal teas. Known as Chai, it is served without milk and there are  types of tea with milk. The most interesting part of having the tea is the setup placed on the table along with instructions on brewing the specific  The tea we had was Kashmiri Kahwa. Expecting a glass of dark brown tea (we had without milk), we were surprised to see an apparatus placed on the table with an instruction card specifying  to  minutes to brew the tea. So we waited for  minutes and pressed the button and lo!, we had  ml of   filtered brewed tea.   We were told we can brew the tea  times with the same ingredients making for  cups of tea when you buy one. Certainly an interesting concept and we were surprised at the cup of aromatic yellow tea. It is not over yet; we were served with Almond bits which we were told is sprinkled as an add on to the Kashmiri Kahwa tea. Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Editors Corner , Food , Restaurants  Comments Tags: Ashok Nagar , chennai , Eggless Cake , Hangout , healthy , Matcha , Salad , snacks , tea , Tea Cafe , Tea Lovers , Tea Trails , Vada Pav , West Mambalam  Moments Epilators make it Super  Fun Feb  Posted by Katie Nathan Hair problems do not end with crowning glory and extends beyond. Hair is wanted and unwanted and it all depends on its location. Imagine a quick call from a friend in the last minute to have fun; it can be an invite from friends, a fun party, and a smoothie with your crush. The first thing that comes to a mind is the time we went for our last shave and the possibility of  All the enthusiasm wanes away and the invitations are no more inviting. Epilators have come a long way and the leader is Braun epilators, researching and specializing on getting silky smooth skin at a moment   s notice! Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Beauty , lifestyle Leave a comment Tags: beauty , Braun , Bridal , chennai , epilators , hair removal , makeup , No shaving , No waxing From Apartments to Villas     Convenience takes  Precedence Jan  Posted by Katie Nathan Casa Grande Pavilion-Phase II Villa Picture Courtesy: Casa Grande This post is about my dream home, a subject that keeps changing its color and style with every passing year. Obviously as new avenues come up, the concept of my dream home takes on a new hue and my plan gets on to works again. So for sure this subject is a living document till I get to the one that meets my requirement with a futuristic vision. It will be quite interesting to know how my concept has changed over the years and what is it that has taken priority today. Having lived in apartments, my interest is now with villas with all the features an apartment would offer. My option for villas is my love for greenery around. Time is such a costly commodity and so maintaining a green cover around has been a challenge. So the project that I would go for should provide  or  bedroom villas with landscaped garden. I would love to pitch in with maintenance now and then for those hands-on times with children. Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in home  Comments Tags: apartments buying , buy villas , casa grande , chennai , houses for sale , own home Godrej Aer Car Fresheners in Six Design  Fragrances Jan  Posted by Katie Nathan Cars are no more a luxury in India  The challenge in maintaining a car lies in regular service and interiors. After all, we spend a lot of time travelling in a closed air conditioned  And if you are the kind of person with motion sickness, the smell of diesel can be quite a challenge to manage. To add to it, we have leather seats and what  Just getting into an ill maintained car is enough to spoil the day for us. With the cabs service picking up big away in Chennai, maintenance has become one of the key factors to get those star ratings from the customers. Car perfumes have a great role to play in keeping the vehicle drive worthy. Though many car fresheners have come into the market, many simply adopt the room freshener style which lasts for an hour and we are back to square one. What we need is a car freshener that keeps the fragrance constant. Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Editors Corner , lifestyle Leave a comment Tags: buy car fresheners online , car fresheners , godrej aer Flower Drum brings Vegetarian Chinese Thai Food to  Adyar Jan  Posted by Katie Nathan Flower drum has been in our hit list for quite some time for its Vegetarian Chinese Thai  Located in Egmore, it was just not forthcoming rather getting distracted to other restaurants once we hit heart of Chennai. Flower Drum opened their other restaurant at Namma Adyar near Aavin circle (above ICICI  We are thankful to Flower Drum and the Chennai food bloggers community to have made it possible to have a firsthand tasting session at this new establishment. First thing we noticed is the ambiance. Few tables were arranged to have a circular seating accommodating  people and more. This has always been an issue when we go as a family for get-togethers. The menu was exhaustive with good varieties highlighting Paneer, Corn, baby corn and  For Tofu lovers, we had few items for you too. Now to the food we had     Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Food , Restaurants Leave a comment Tags: Adyar restaurant , chennai , chinese food , chinese restaurant , flower drum , restaurant review , review , thai food , vegetarian Going Natural: Good Knight Offers Mosquito Repellent Solutions with Citronella and Eucalyptus  Oils Dec  Posted by Katie Nathan Winter is in and this spells onset of mosquitoes. It is a known fact that mosquitoes are vectors responsible for many diseases. As always prevention is better than cure and it is important for us to take necessary steps to secure us from potential mosquito bites. Immunity is a major factor in keeping away seasonal diseases communicated by various medium. When we talk about immunity, babies and children are the most susceptible and as a parent every one of us know how difficult it is to control them and keep them away from places where mosquitoes breed. The next aspect of immunity is allergic reactions. Today, allergies are the cause of many respiratory associated infections and so we hear    natural    being chanted everywhere and in each item we purchase. It is not that everyone is allergic to chemicals but we can say that there is a formidable population in that area too. Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Editors Corner , Health , lifestyle  Comment Tags: citronella , eucalyptus , fabric rollon , good knight , mosquito , natural prevention , protection for babies , repellent , vectors Sensodyne Food Walk      Chennai Dec  Posted by Katie Nathan A group of bloggers, including me, were invited for the So Chennai & Sensodyne Food Walk here in Chennai. Needless to say, I grabbed the offer and set out for the great food-adventure! The concept at hand was that a few bloggers and actual tooth sensitivity sufferers, who had been using Sensodyne for a number of months, would get to go on a food walk, sampling food and drinks without the fear of any sensitivity. Here is a description of our food-adventure experience at three popular eateries in Chennai. Read the rest of this entry     Share this: Tumblr Twitter Facebook LinkedIn Google Pinterest Reddit Pocket More Email Print Like this: Like  Posted in Food , Health , lifestyle  Comments Tags: chennai , healthy lifestyle , sensodyne , tingling pain in mouth , tooth pain , tooth sensitivity , unable to eat cold things     Older Posts Looking for a Specific Post ? Search for: Topics Art  Attractions  Beauty  Business  Ecological Restoration in Chennai  Editors Corner  Education  Entertainment  Events  Fashion  Finance  Food  Restaurants  Gardening  Health  Heritage of Chennai  Hobby  Book Reviews  home  lifestyle  Media  Medical  Mystic Traditions MyTra  parenting  Pets  Sports  Adventure  Stay  Technology  Transportation  Travel  Watch Out  Our Latest Posts Welcome to Era of Easy Communication and Misleading  Information Review of House of Cards Season   Mango Madness at The Forum Vijaya  Mall Copper Chimney   s The Great Indian Kabab Festival     Dig into Authentic Traditional  Food Travel Diary      Travel to Madhya Pradesh Jyotir Lingams      Ujjain Chennai Focus on Viber Chennai Focus has its own Public Chat in Viber. Connect with us if you are in Viber - if not, check out Chennai Focus on Viber Chennai Focus Pitch Video Watch Chennai Focus Pitch Video Here. Share and Show your Support to us. Become a Eco-Tourist Thanks Tripoto Thanks Zomato Thanks Indiblogger    Thanks Blogmint Thanks Blogadda Thanks Indian Top Blogs Thanks SHeroes Archives Archives Select Month June   May   March   February   January   December   November   October   September   July   June   May   April   March   February   January   December   November   October   September   August   July   June   May   April   March   February   November   September   August   July   June   May   April   March   February   January   December   November   Top Posts & Pages Know Our Own Dog Breeds, Kombai, Kanni, Alangu, Chippiparai and Rajapalayam Healthy Meal Tiffin Services Delivered at Homes and Offices Blog Interactions Mukti Patel on Summer Collection Special from    Katie Nathan on Travel : Indulge in a Spiritua    Siva on Travel : Indulge in a Spiritua    MG on Healthy Meal Tiffin Services D    Siva on Blog Disclosure Statement Follow Blog via Email Enter your email address to follow this blog and receive notifications of new posts by email. Join  other followers Facebook Facebook Tweeting Now RT @  : #nombukanji  #TamilNadu #tradition #breaking #fast #healthy #onepotmeal #loaded with #goodness @ FoodBloggerAI @     Welcome to Era of Easy Communication and Misleading Information  on @ bloglovin  Welcome to Era of Easy Communication and Misleading Information  #PlasticRice #Food #Chennai #Blog #India #Tamilnadu  Welcome to Era of Easy Communication and Misleading  Information    A cup of hot choco  f d\\\\xab laced coffee    \\\\xef  f shared with a long lost friend is blissfully tasted and time well spent      Follow @chennaifocus We Support On Instagram Posts I Like TATA TIGOR - STYLE BACK (#Tigo    on MultitaskingMarwari Goodreads Blog Promotion Follow my blog with Bloglovin Blog Content Marketing Blog Content Marketing Blog Content Marketing Our Visitors  visits and counting. Thanks Friends! Blog at   Send to Email Address Your Name Your Email Address  = \\\'\\\'; Cancel Post was not sent - check your email addresses! Email check failed, please try again Sorry, your blog cannot share posts by email. Post to Cancel    %d bloggers like this:  "  \''

In [151]:
naivebayes.predict(tfidf_trans.transform(vect.transform(tokenize(example_blog_post))))

array([False], dtype=bool)

In [297]:
naivebayes = MultinomialNB()

In [77]:
target = df['quality']

In [78]:
naivebayes.fit(vectorized_tokens, target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [179]:
data = df['text']

In [152]:
predicted = naivebayes.predict(tfidf_trans.transform(vect.transform(tokens)))

In [80]:
# predicted = naivebayes.predict(vect.transform(data))

In [153]:
predicted = pd.DataFrame(predicted)

In [82]:
target.value_counts()

False    2836
True      977
Name: quality, dtype: int64

In [154]:
predicted[0].value_counts()

False    2930
True      883
Name: 0, dtype: int64

In [276]:
# df.drop('index',axis=1, inplace=True)

In [108]:
# df.drop('predicted',axis=1, inplace=True)

# df.drop('index',axis=1, inplace=True)

In [85]:
# predicted.shape

In [186]:
df = df.join(predicted)

In [187]:
df.rename(columns = {0:'predicted'}, inplace=True)

In [180]:
# predicted = df['predicted']
target = df['quality']

In [181]:
# Max features = 2000
print(metrics.classification_report(target, predicted))

             precision    recall  f1-score   support

      False       0.88      0.91      0.89      2836
       True       0.70      0.63      0.66       977

avg / total       0.83      0.84      0.83      3813



In [182]:
df['url'] = df['url'].apply(remove_tag_category_pages)
df.dropna(inplace=True)

df.reset_index(inplace=True)

df.drop('index',axis=1,inplace=True)

In [183]:
predicted = df['predicted']

KeyError: 'predicted'

In [184]:
target = df['quality']

In [2]:
# Max features = 2000
print(metrics.classification_report(target, predicted))

In [147]:
# Max features = 2000
print(metrics.classification_report(target, predicted))

             precision    recall  f1-score   support

      False       0.87      0.92      0.89      2836
       True       0.72      0.60      0.65       977

avg / total       0.83      0.84      0.83      3813



In [149]:
# data2 = withheld['text']
# target2 = withheld['quality']

In [ ]:
predicted2 = test_pipeline.predict(data2)

In [ ]:
predicted2 = pd.DataFrame(predicted2)

In [ ]:
# Max features = 2000
print(metrics.classification_report(target, predicted))

In [ ]:
withheld = withheld.join(predicted2)

In [ ]:
withheld.rename(columns = {0:'predicted'}, inplace=True)

In [ ]:
withheld.query('predicted == True').shape

In [ ]:
metrics.classification_report?

In [ ]:
# Max features = 2500
print(metrics.classification_report(target, predicted))

In [342]:
pickle.dump( naivebayes, open( "fit_naive_bayes.p", "wb" ) )

In [6]:
naivebayes = pickle.load(open("fit_naive_bayes.p", "rb"))

## Explore Assignments

In [218]:
true_positive = df.query("predicted == True").query("quality == True")

In [219]:
true_positive.shape

(449, 5)

In [220]:
true_positive.reset_index(inplace=True)

In [221]:
true_positive.drop('index',axis=1,inplace=True)

In [222]:
true_positive[:50]

,_id,quality,text,url,predicted
0,593edbbd29d4b520f09f7987,True,b'b\' ACM Sign In Search Go Join ACM About C...,https://cacm.acm.org/blogs/blog-cacm/169199-da...,True
1,593edbbe29d4b520f09f7988,True,b'b\'CoolData blog December Amazing things w...,https://cooldata.wordpress.com/2016/12/05/amaz...,True
2,593edbbe29d4b520f09f7989,True,b'b\'CoolData blog July Getting bitten by P...,https://cooldata.wordpress.com/2013/07/30/gett...,True
3,593edbbf29d4b520f09f798a,True,b'b\'CoolData blog April Data I want to play...,https://cooldata.wordpress.com/2012/04/24/data...,True
4,593edbbf29d4b520f09f798b,True,"b'b\'CoolData blog February Data disasters, ...",https://cooldata.wordpress.com/2011/02/22/data...,True
5,593edbc029d4b520f09f798c,True,b'b\'CoolData blog January New finds in old ...,https://cooldata.wordpress.com/2015/01/07/new-...,True
6,593edbc029d4b520f09f798d,True,b'b\'CoolData blog September What predictor ...,https://cooldata.wordpress.com/2014/09/22/what...,True
7,593edbc129d4b520f09f798e,True,b'b\'CoolData blog June Time management for ...,https://cooldata.wordpress.com/2013/06/27/time...,True
8,593edbc129d4b520f09f798f,True,b'b\'CoolData blog August Exploring associat...,https://cooldata.wordpress.com/2015/08/26/expl...,True
9,593edbc129d4b520f09f7990,True,b'b\'Data Science Search Primary Menu Skip to...,http://101.datascience.community/2016/09/22/da...,True


In [97]:
false_positive = df.query('predicted == True').query('quality == False')

In [98]:
false_positive.reset_index(inplace=True)

In [99]:
false_positive.drop('index',axis=1,inplace=True)

In [100]:
false_positive.shape

(205, 5)

In [106]:
false_positive[200:]

,_id,quality,text,url,predicted
200,593f8adbf7e49c258ed37fdd,False,b'b\'Bandit Beta Rodeo New ScienceOps Company ...,http://blog.yhat.com/posts/rodeo-2.3.0.html,True
201,593f8addf7e49c258ed37fde,False,b'b\'Bandit Beta Rodeo New ScienceOps Company ...,http://blog.yhat.com/posts/rodeo-2.0.6-datafra...,True
202,593f51e0f7e49c1f7a2906a1,False,b'b\'Junk Charts Recycling chartjunk as junk a...,http://junkcharts.typepad.com/junk_charts/2017...,True
203,593f8621f7e49c258ed37f2e,False,b'b\'Junk Charts Recycling chartjunk as junk a...,http://junkcharts.typepad.com/junk_charts/2016...,True
204,593f8545f7e49c258ed37f1b,False,b'b\'Junk Charts Recycling chartjunk as junk a...,http://junkcharts.typepad.com/junk_charts/2016...,True


In [204]:
false_negative = df.query('predicted == False').query('quality==True')

In [205]:
false_negative.reset_index(inplace=True)

In [206]:
false_negative.drop('index',axis=1,inplace=True)

In [207]:
false_negative.shape

(378, 5)

In [208]:
false_negative[:50]

,_id,quality,text,url,predicted
0,593edc1329d4b520f09f79eb,True,"b' Too Many Requests @ Mon, Jun GMT '",http://www.juiceanalytics.com/writing/the-futu...,False
1,593edbbd29d4b520f09f7987,True,b' ACM Sign In Search Go Join ACM About Comm...,https://cacm.acm.org/blogs/blog-cacm/169199-da...,False
2,593edbbf29d4b520f09f798a,True,b'CoolData blog April Data I want to play w...,https://cooldata.wordpress.com/2012/04/24/data...,False
3,593edbc329d4b520f09f7999,True,"b""Toggle navigation Biography Blog Contact Ne...",http://inverseprobability.com/2017/03/14/data-...,False
4,593edbc329d4b520f09f799d,True,"b""Toggle navigation Biography Blog Contact Ne...",http://inverseprobability.com/2016/02/29/futur...,False
5,593edbc329d4b520f09f799e,True,"b""Toggle navigation Biography Blog Contact Ne...",http://inverseprobability.com/2016/01/17/machi...,False
6,593edbc329d4b520f09f799f,True,"b""Toggle navigation Biography Blog Contact Ne...",http://inverseprobability.com/2016/01/17/machi...,False
7,593edbc429d4b520f09f79a4,True,"b""Toggle navigation Biography Blog Contact Ne...",http://inverseprobability.com/2015/12/14/opena...,False
8,593edbc429d4b520f09f79a5,True,"b""Toggle navigation Biography Blog Contact Ne...",http://inverseprobability.com/2016/05/29/data-...,False
9,593edbc429d4b520f09f79a6,True,"b""Toggle navigation Biography Blog Contact Ne...",http://inverseprobability.com/2016/06/03/data-...,False


In [56]:
true_negative = df.query('predicted == False').query('quality == False')

In [57]:
true_negative.reset_index(inplace=True)
true_negative.drop('index',axis=1,inplace=True)

In [58]:
true_negative.shape

(2403, 5)

In [62]:
df['predicted'].value_counts()

False    2811
True      444
Name: predicted, dtype: int64

In [63]:
df['quality'].value_counts()

False    2457
True      798
Name: quality, dtype: int64

Play with countvect, add in treebank tokenizer, compare tf-idf to no tf-idf....